In [1]:
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from collections import Counter
from IPython.display import clear_output

In [2]:
def eval_model(model, test_data, test_labels):
    predictions = model.predict(test_data)
    #print(predictions)
    predictions = list(np.around(np.array(predictions),0))
    accuracy = accuracy_score(test_labels, predictions)
    #print(accuracy)
    return accuracy

In [3]:
def other_class(n_classes, current_class):
    """
    Returns a list of class indices excluding the class indexed by class_ind
    :param nb_classes: number of classes in the task
    :param class_ind: the class index to be omitted
    :return: one random class that != class_ind
    """
    #print(current_class)
    if current_class < 0 or current_class >= n_classes:
        error_str = "class_ind must be within the range (0, nb_classes - 1)"
        raise ValueError(error_str)

    other_class_list = list(range(n_classes))
    other_class_list.remove(current_class)
    other_class = np.random.choice(other_class_list)
    return other_class

def inject_noise(n_classes, y_, noise_level):
    y = y_.copy()
    if noise_level > 100 or noise_level < 0:
        raise ValueError('Noise level can not be bigger than 100 or smaller than 0')

    noisy_idx = np.random.choice(len(y), int(len(y)*noise_level/100.0), replace = False)
    for i in noisy_idx:
        y[i] = other_class(n_classes, y[i])

    return y

In [4]:
# Load testing data
dataset = "tasks-quarter-original"
test_labels = np.loadtxt("data/" + dataset + "-test-labels.txt")[0:6000]
test_data = np.loadtxt("data/" + dataset + "-test-data.txt")[0:6000]
# Load training data
train_labels = np.loadtxt("data/" + dataset + "-train-labels.txt")
train_data = np.loadtxt("data/" + dataset + "-train-data.txt")
n_classes = len(list(set(list(train_labels))))

In [5]:
# this preprocessing step is needed because original task data classes are [2,3,4,5], this steps to make the classes to [0,1,2,3]
for i in range(train_labels.shape[0]):
    train_labels[i] = train_labels[i] - 2
for j in range(test_labels.shape[0]):
    test_labels[j] = test_labels[j] - 2

In [6]:
print(Counter(list(train_labels)).keys()) # equals to list(set(words))
print(Counter(list(train_labels)).values()) # counts the elements' frequency

[0.0, 1.0, 2.0, 3.0]
[3, 38, 10825, 3134]


In [7]:
print(Counter(list(test_labels)).keys()) # equals to list(set(words))
print(Counter(list(test_labels)).values()) # counts the elements' frequency

[1.0, 2.0, 3.0]
[11, 4644, 1345]


In [8]:
test_data.shape

(6000, 26)

In [9]:
train_data.shape

(14000, 26)

KNN task test, 10 time averaged results are showed in the end of output

In [41]:
# KNN 
knnmodel = KNeighborsClassifier(n_neighbors=4, weights = 'distance')
acc_list_knn = []
repeat = 10
for i in range(repeat):
    for noise in [0,10,20,30,40,50,60,70,80,90,100]:
        train_noisy_labels = inject_noise(n_classes, train_labels, noise)
        knnmodel.fit(train_data, train_noisy_labels)
        acc = eval_model(knnmodel, test_data, test_labels)
        acc_list_knn.append(acc)
        print("noise_level, accuracy", noise, acc)
avg_result = np.zeros(11)
for i in range(repeat):
    for j in range(11):
        avg_result[j] += acc_list_knn[j+i*11]
average = avg_result/repeat
print(average)

('noise_level, accuracy', 0, 0.8676666666666667)
('noise_level, accuracy', 10, 0.8081666666666667)
('noise_level, accuracy', 20, 0.7501666666666666)
('noise_level, accuracy', 30, 0.6591666666666667)
('noise_level, accuracy', 40, 0.5721666666666667)
('noise_level, accuracy', 50, 0.48083333333333333)
('noise_level, accuracy', 60, 0.40016666666666667)
('noise_level, accuracy', 70, 0.2881666666666667)
('noise_level, accuracy', 80, 0.20983333333333334)
('noise_level, accuracy', 90, 0.11633333333333333)
('noise_level, accuracy', 100, 0.043333333333333335)
('noise_level, accuracy', 0, 0.8676666666666667)
('noise_level, accuracy', 10, 0.807)
('noise_level, accuracy', 20, 0.7396666666666667)
('noise_level, accuracy', 30, 0.6671666666666667)
('noise_level, accuracy', 40, 0.5746666666666667)
('noise_level, accuracy', 50, 0.49366666666666664)
('noise_level, accuracy', 60, 0.3888333333333333)
('noise_level, accuracy', 70, 0.29983333333333334)
('noise_level, accuracy', 80, 0.21216666666666667)
('noi

MLP task test, 10 time averaged results are showed in the end of output

In [42]:
# MLP task
mlpmodel = MLPClassifier(solver='adam', hidden_layer_sizes=(28,28), random_state=1)
acc_list_mlp = []
repeat = 10
for i in range(repeat):
    for noise in [0,10,20,30,40,50,60,70,80,90,100]:
        train_noisy_labels = inject_noise(n_classes, train_labels, noise)
        mlpmodel.fit(train_data, train_noisy_labels)
        acc = eval_model(mlpmodel, test_data, test_labels)
        acc_list_mlp.append(acc)
        print("noise_level, accuracy", noise, acc)
avg_result = np.zeros(11)
for i in range(repeat):
    for j in range(11):
        avg_result[j] += acc_list_mlp[j+i*11]
average = avg_result/repeat
print(average)

/anaconda3/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


('noise_level, accuracy', 0, 0.8346666666666667)
('noise_level, accuracy', 10, 0.8263333333333334)
('noise_level, accuracy', 20, 0.8196666666666667)
('noise_level, accuracy', 30, 0.801)
('noise_level, accuracy', 40, 0.7956666666666666)
('noise_level, accuracy', 50, 0.7443333333333333)
('noise_level, accuracy', 60, 0.643)
('noise_level, accuracy', 70, 0.4563333333333333)
('noise_level, accuracy', 80, 0.13916666666666666)
('noise_level, accuracy', 90, 0.04783333333333333)
('noise_level, accuracy', 100, 0.014333333333333333)
('noise_level, accuracy', 0, 0.8346666666666667)
('noise_level, accuracy', 10, 0.8296666666666667)
('noise_level, accuracy', 20, 0.8223333333333334)
('noise_level, accuracy', 30, 0.7966666666666666)
('noise_level, accuracy', 40, 0.7923333333333333)
('noise_level, accuracy', 50, 0.7315)
('noise_level, accuracy', 60, 0.6655)
('noise_level, accuracy', 70, 0.4205)
('noise_level, accuracy', 80, 0.16466666666666666)
('noise_level, accuracy', 90, 0.043666666666666666)
('nois

NearestCentroid task test, as the results varies a lot from each run, 100 time averaged results are showed in the end of output

In [40]:
# NearestCentroid task
ncmodel = NearestCentroid()
acc_list_nc = []
repeat = 100
for i in range(repeat):
    for noise in [0,10,20,30,40,50,60,70,80,90,100]:
        train_noisy_labels = inject_noise(n_classes, train_labels, noise)
        ncmodel.fit(train_data, train_noisy_labels)
        acc = eval_model(ncmodel, test_data, test_labels)
        acc_list_nc.append(acc)
        print("noise_level, accuracy", noise, acc)
avg_result = np.zeros(11)
for i in range(repeat):
    for j in range(11):
        avg_result[j] += acc_list_nc[j+i*11]
average = avg_result/repeat
print(average)
            

('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.35783333333333334)
('noise_level, accuracy', 20, 0.24566666666666667)
('noise_level, accuracy', 30, 0.4225)
('noise_level, accuracy', 40, 0.32616666666666666)
('noise_level, accuracy', 50, 0.4073333333333333)
('noise_level, accuracy', 60, 0.23883333333333334)
('noise_level, accuracy', 70, 0.24166666666666667)
('noise_level, accuracy', 80, 0.2618333333333333)
('noise_level, accuracy', 90, 0.242)
('noise_level, accuracy', 100, 0.23583333333333334)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.3478333333333333)
('noise_level, accuracy', 20, 0.3993333333333333)
('noise_level, accuracy', 30, 0.3963333333333333)
('noise_level, accuracy', 40, 0.269)
('noise_level, accuracy', 50, 0.38283333333333336)
('noise_level, accuracy', 60, 0.2828333333333333)
('noise_level, accuracy', 70, 0.26516666666666666)
('noise_level, accuracy', 80, 0.23466666666666666)
('noise_level, accuracy', 90, 0.2811666666666667)
('no

('noise_level, accuracy', 70, 0.19066666666666668)
('noise_level, accuracy', 80, 0.228)
('noise_level, accuracy', 90, 0.23483333333333334)
('noise_level, accuracy', 100, 0.21216666666666667)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.4251666666666667)
('noise_level, accuracy', 20, 0.42533333333333334)
('noise_level, accuracy', 30, 0.4216666666666667)
('noise_level, accuracy', 40, 0.268)
('noise_level, accuracy', 50, 0.3923333333333333)
('noise_level, accuracy', 60, 0.19983333333333334)
('noise_level, accuracy', 70, 0.18933333333333333)
('noise_level, accuracy', 80, 0.17083333333333334)
('noise_level, accuracy', 90, 0.26366666666666666)
('noise_level, accuracy', 100, 0.26916666666666667)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.4081666666666667)
('noise_level, accuracy', 20, 0.415)
('noise_level, accuracy', 30, 0.49383333333333335)
('noise_level, accuracy', 40, 0.332)
('noise_level, accuracy', 50, 0.30283333333333334)
('noise_level, 

('noise_level, accuracy', 70, 0.3323333333333333)
('noise_level, accuracy', 80, 0.2405)
('noise_level, accuracy', 90, 0.2653333333333333)
('noise_level, accuracy', 100, 0.2)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.388)
('noise_level, accuracy', 20, 0.36583333333333334)
('noise_level, accuracy', 30, 0.37883333333333336)
('noise_level, accuracy', 40, 0.4185)
('noise_level, accuracy', 50, 0.3805)
('noise_level, accuracy', 60, 0.3451666666666667)
('noise_level, accuracy', 70, 0.2355)
('noise_level, accuracy', 80, 0.21216666666666667)
('noise_level, accuracy', 90, 0.2075)
('noise_level, accuracy', 100, 0.217)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.3955)
('noise_level, accuracy', 20, 0.4728333333333333)
('noise_level, accuracy', 30, 0.2555)
('noise_level, accuracy', 40, 0.3333333333333333)
('noise_level, accuracy', 50, 0.38433333333333336)
('noise_level, accuracy', 60, 0.43483333333333335)
('noise_level, accuracy', 70, 0.362)
('noise

('noise_level, accuracy', 70, 0.2871666666666667)
('noise_level, accuracy', 80, 0.3293333333333333)
('noise_level, accuracy', 90, 0.22816666666666666)
('noise_level, accuracy', 100, 0.2205)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.3885)
('noise_level, accuracy', 20, 0.44483333333333336)
('noise_level, accuracy', 30, 0.4)
('noise_level, accuracy', 40, 0.43666666666666665)
('noise_level, accuracy', 50, 0.427)
('noise_level, accuracy', 60, 0.37833333333333335)
('noise_level, accuracy', 70, 0.3481666666666667)
('noise_level, accuracy', 80, 0.14516666666666667)
('noise_level, accuracy', 90, 0.235)
('noise_level, accuracy', 100, 0.2911666666666667)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.46)
('noise_level, accuracy', 20, 0.4013333333333333)
('noise_level, accuracy', 30, 0.38433333333333336)
('noise_level, accuracy', 40, 0.4076666666666667)
('noise_level, accuracy', 50, 0.4345)
('noise_level, accuracy', 60, 0.25466666666666665)
('noise_

('noise_level, accuracy', 70, 0.395)
('noise_level, accuracy', 80, 0.25566666666666665)
('noise_level, accuracy', 90, 0.2831666666666667)
('noise_level, accuracy', 100, 0.22133333333333333)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.39716666666666667)
('noise_level, accuracy', 20, 0.41633333333333333)
('noise_level, accuracy', 30, 0.4056666666666667)
('noise_level, accuracy', 40, 0.468)
('noise_level, accuracy', 50, 0.23416666666666666)
('noise_level, accuracy', 60, 0.3908333333333333)
('noise_level, accuracy', 70, 0.36833333333333335)
('noise_level, accuracy', 80, 0.33066666666666666)
('noise_level, accuracy', 90, 0.24916666666666668)
('noise_level, accuracy', 100, 0.2475)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.33266666666666667)
('noise_level, accuracy', 20, 0.2753333333333333)
('noise_level, accuracy', 30, 0.41383333333333333)
('noise_level, accuracy', 40, 0.32016666666666665)
('noise_level, accuracy', 50, 0.2605)
('noise_level

('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.404)
('noise_level, accuracy', 20, 0.3908333333333333)
('noise_level, accuracy', 30, 0.4025)
('noise_level, accuracy', 40, 0.277)
('noise_level, accuracy', 50, 0.4101666666666667)
('noise_level, accuracy', 60, 0.2613333333333333)
('noise_level, accuracy', 70, 0.173)
('noise_level, accuracy', 80, 0.23766666666666666)
('noise_level, accuracy', 90, 0.22633333333333333)
('noise_level, accuracy', 100, 0.232)
('noise_level, accuracy', 0, 0.5345)
('noise_level, accuracy', 10, 0.30216666666666664)
('noise_level, accuracy', 20, 0.44483333333333336)
('noise_level, accuracy', 30, 0.3473333333333333)
('noise_level, accuracy', 40, 0.4038333333333333)
('noise_level, accuracy', 50, 0.5243333333333333)
('noise_level, accuracy', 60, 0.15216666666666667)
('noise_level, accuracy', 70, 0.3965)
('noise_level, accuracy', 80, 0.20416666666666666)
('noise_level, accuracy', 90, 0.25866666666666666)
('noise_level, accuracy', 100, 0.2765)
('noi